# <font color='dimgrey'>Routing : Direct Exchange </font>


## <font color='grey'> Till now ... </font>
In the last tutorial 'Fanout Exchange' We had producer sending messages via Fanout exchange pushes to all the queues

<font color='dimgrey'> In this tutorial  </font>
> [Queues are going to subscribe to a subset of messages](https://www.rabbitmq.com/tutorials/tutorial-four-python.html)

<font color='dimgrey'> In Fanout exchange we used to have queues binded to the exchanges. In this one we will also have a 'routing_key' parameter which subscribes to messages of same 'routing_key' while publishing it </font>

<font color='dimgrey'> <h2> Publishing </h2></font> <font color='dimgrey'>  Sending messages would have an extra 'routing_key' parameter specifying the kind of message to recieve from the exchange </font>

In [ ]:
channel.basic_publish(exchange='direct_logs',
                      routing_key=severity,
                      body=message)

<font color='dimgrey'> <h2> Subscribing </h2></font> <font color='dimgrey'>  New exchange-queue  binding will look like below </font>

In [ ]:
channel.queue_bind(exchange=exchange_name,
                   queue=queue_name,
                   routing_key='black')

<font color='dimgrey'> Instead of </font>

In [ ]:
channel.queue_bind(exchange=exchange_name,
                   queue=queue_name)

<font color='dimgrey'> In the case of logs, we might have different severity of messages like 'info', 'warning' or 'error'. While some queue needs all the messages to display on a screen while some needs to only store most severe 'error' messages. Direct exchange are gonna let us do just that </font>

<font color='dimgrey'> <h3> Subscribing </h3> to more than one message type would look like following: </h3></font>

In [ ]:
result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

for severity in severities:
    channel.queue_bind(exchange='direct_logs',
                       queue=queue_name,
                       routing_key=severity)

<font color='dimgrey'> We create multiple bindings with the same exchange and queue for each 'routing_key' type. In our case - severity of the log messages </font>

<font color='dimgrey'> <h2> Final Code : Publishing </h2> </font>

In [ ]:
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='direct_logs', exchange_type='direct')

def sendQueue():
    try:
        while True:
            # keep taking input till kernel is interrupted
            input_ = input("Enter new task")
            severity, message = input_.split()
            channel.basic_publish(exchange='direct_logs', routing_key=severity, body=message)
            print(" [x] Sent %r:%r" % (severity, message))
    except KeyboardInterrupt:
        # Exit gracefully
        connection.close()
sendQueue()

<font color='dimgrey'> <h2> Final Code : Subscribing </h2> </font>

In [ ]:
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='direct_logs', exchange_type='direct')

result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

severities = input().split()
if not severities:
    sys.stderr.write("Usage: %s [info] [warning] [error]\n" % severities[0])
    sys.exit(1)

for severity in severities:
    channel.queue_bind(
        exchange='direct_logs', queue=queue_name, routing_key=severity)

print(' [*] Waiting for logs. To exit kernel -> Interrupt')


def callback(ch, method, properties, body):
    print(" [x] %r:%r" % (method.routing_key, body))


channel.basic_consume(
    queue=queue_name, on_message_callback=callback, auto_ack=True)

try:
    channel.start_consuming()
except KeyboardInterrupt:
    # Exit gracefully
    connection.close()

## <font color='dimgrey'> Testing </font> 
<t><font color='dimgrey'> We will run publisher code from this notebook and we will create two worker nodes for consuming. We will run the below code after subscriber nodes have declared queues </font> 

In [3]:
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='direct_logs', exchange_type='direct')

def sendQueue():
    try:
        while True:
            # keep taking input till kernel is interrupted
            severity, message = input("Enter new task").split(' ', 1)
            channel.basic_publish(exchange='direct_logs', routing_key=severity, body=message)
            print(" [x] Sent %r:%r" % (severity, message))
    except KeyboardInterrupt:
        # Exit gracefully
        connection.close()
sendQueue()

Enter new taskerror "error message"
 [x] Sent 'error':'"error message"'
Enter new taskinfo "info message"
 [x] Sent 'info':'"info message"'


#### <font color='dimgrey'> As you can see subscriber node 1 recieved only error message as it's subscribed to error and warning message kinds only where as subscriber node 2 has recived both error and info messages. Message of type warning will also be recieved by both the subscribers </font> 

In [ ]:
>  <font color='dimgrey'> If every queue subscribes to every kind of message in a direct exchange then that would mean the same thing as using a fanout exchange </font> 